# NEW

## Setup

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# import sagemaker

# sess = sagemaker.Session()
# role = sagemaker.get_execution_role()

## Download and store data locally

In [1]:
import numpy as np
import h5py
import os
from keras.datasets import fashion_mnist

# load data
(X_train, Y_train), (X_val, Y_val) = fashion_mnist.load_data()

Using TensorFlow backend.
/anaconda3/envs/fashion/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/envs/fashion/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/envs/fashion/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/envs/fashion/lib/python3.7/site-packages/t

In [4]:
# create directory for data
os.makedirs("./data", exist_ok = True)

# store in hdf5 files
with h5py.File('./data/train.h5', 'w') as hf:
    hf.create_dataset('X_train', data=X_train)
    hf.create_dataset('Y_train', data=Y_train)

with h5py.File('./data/val.h5', 'w') as hf:
    hf.create_dataset('X_val', data=X_val)
    hf.create_dataset('Y_val', data=Y_val)

## Upload data to s3

In [ ]:
prefix = 'keras-cnn-fashion-mnist'

training_input_path   = sess.upload_data('data/training.h5', key_prefix=prefix+'/training')
validation_input_path = sess.upload_data('data/validation.h5', key_prefix=prefix+'/validation')

print(training_input_path)
print(validation_input_path)

## Train on GPU spot instance using automatic tuning

In [ ]:


# tensorflow estimator

# hyperparameter spaces

# tuner

## Deploy best model

## Cleanup

# OLD

## Load and prepare the data

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import warnings


from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, Flatten, Conv2D
from keras.layers import MaxPooling2D, Dropout
from keras.models import Model, load_model
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.regularizers import l2
from keras.datasets import fashion_mnist


from sklearn.model_selection import train_test_split

%matplotlib inline
warnings.simplefilter(action='ignore', category=FutureWarning)
np.random.seed(27)

Using TensorFlow backend.
/anaconda3/envs/digits/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/envs/digits/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/envs/digits/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/envs/digits/lib/python3.7/site-packages/tenso

In [22]:
# load csvs into dataframes
(X_train, Y_train), (X_val, Y_val) = fashion_mnist.load_data()

In [23]:
# reshape for keras
X_train = X_train.reshape(-1, 28, 28, 1)
X_val = X_val.reshape(-1, 28, 28, 1)
X_train.shape

(60000, 28, 28, 1)

In [24]:
# normalize
X_train = X_train / 255.0
X_val = X_val / 255.0

In [25]:
# encode labels
Y_train = to_categorical(Y_train, num_classes = 10)
Y_val = to_categorical(Y_val, num_classes = 10)

In [26]:
Y_train.shape

(60000, 10)

## Create model

In [27]:
def DigitsModel(input_shape):
    """
    Keras model for Fashion MNIST data.
    
    Parameters
    ---------
    input_shape: tuple
        Shape of image inputs
        
    Returns
    -------
    model: keras.Model()
        Compiled model for image data, not yet fit.
    
    """
    # Input placeholder
    X_input = Input(input_shape)

    # Block 1 - convolution
    X = ZeroPadding2D((1, 1))(X_input)
    X = Conv2D(32, (4, 4), strides = (1, 1), name = 'conv0')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='max_pool0')(X)
    
    # Block 2 - convolution
    X = ZeroPadding2D((1, 1))(X)
    X = Conv2D(48, (3, 3), strides = (2, 2), name = 'conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), padding='same', name='max_pool1')(X)
    
    # Block 3 - convolution
    X = ZeroPadding2D((1, 1))(X)
    X = Conv2D(64, (2, 2), strides = (2, 2), name = 'conv2')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), padding='same', name='max_pool2')(X)
    
    # Block 4 - flattened, fully connected
    X = Flatten()(X)
    X = Dense(128, activation='relu', name='fc3')(X)
    X = Dense(10, activation='softmax', name='fc4')(X)
    
    # create model
    model = Model(inputs = X_input, outputs = X, name='DigitsModel')
    
    # compile
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

## Data augmentation

In [28]:
datagen = ImageDataGenerator(
    rotation_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=0.1)

datagen.fit(X_train)

## Train model

In [29]:
# model instance
week8model = DigitsModel((28, 28, 1))

In [30]:
# callbacks
check = ModelCheckpoint(filepath='weights.{epoch:02d}-{val_acc:.4f}.hdf5',
                        monitor='val_acc', save_best_only=True)
stop = EarlyStopping(monitor='val_acc', patience=6)
lr_reduce = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=3)
callbacks = [check, stop, lr_reduce]

In [31]:
epochs, batch_size = 40, 32
hist = week8model.fit_generator(datagen.flow(X_train, Y_train, batch_size = batch_size),
                                callbacks=callbacks, epochs=epochs, 
                                validation_data = (X_val,Y_val),
                                verbose = 2, 
                                steps_per_epoch=X_train.shape[0] // batch_size)

Epoch 1/40
 - 49s - loss: 0.6957 - acc: 0.7380 - val_loss: 0.5010 - val_acc: 0.8169
Epoch 2/40
 - 49s - loss: 0.4960 - acc: 0.8132 - val_loss: 0.4348 - val_acc: 0.8352
Epoch 3/40
 - 61s - loss: 0.4390 - acc: 0.8362 - val_loss: 0.3778 - val_acc: 0.8584
Epoch 4/40
 - 72s - loss: 0.4000 - acc: 0.8500 - val_loss: 0.3434 - val_acc: 0.8727
Epoch 5/40
 - 103s - loss: 0.3757 - acc: 0.8583 - val_loss: 0.3375 - val_acc: 0.8746
Epoch 6/40
 - 96s - loss: 0.3595 - acc: 0.8626 - val_loss: 0.3174 - val_acc: 0.8816
Epoch 7/40


KeyboardInterrupt: 

## Evaluate

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].plot(hist.history['loss'], color='b', label="Training loss")
ax[0].plot(hist.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(hist.history['acc'], color='b', label="Training accuracy")
ax[1].plot(hist.history['val_acc'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)


## Submit

In [ ]:
submit_model = load_model('kaggle_weights.03-0.9940.hdf5')
test_preds = submit_model.predict(x=X_test).argmax(axis=-1)
submission_df = pd.DataFrame({'ImageId': np.arange(1, test.shape[0] + 1), 'Label': test_preds})
submission_df.to_csv('submit.csv', index=False)